AutoRegression (AR), Integrated (I), Moving Average (MA) components form ARIMA model. 

AR part indicates observed valued is dependent on lagged values. 

MA component indicates relationship exists between current value and linear combination of prior stochastic terms. Each stochastic term is sampled from white noise which is a stationary random process that samples from a normal distribution with mean zero. Note MA in ARIMA isn't the same as the MA filter which smooths out a time series. Each stochastic term from white noise is independent and represents random shocks or error terms in the process (data we cannot account for). 

I component refers to number of differencing levels required to make series stationary to apply AR, MA.

Non-seasonal ARIMA models denoted ARIMA(p, d, q) where p, d, q are non-negative integers. Where p refers to order of AR, d is differencing levels needed, and q is order of MA. 

ARIMA models are a additivite combination of each component and solving for the parameters of each component allows for forecasting following time point. 

Denote observed outcomes by O, and the set of parameters that describe some stochastic process by theta. Therefore to compute probability of an event, do: P(O | theta). Meaning given some parameters we can figure out the probability of observing O. 

However in reality it is the opposite, where we are given observations O and want to estimate the parameters to obtain a model to use for forecasting or prediction. In that case, we want to choose parameters that maximize the likelihood that we get observations O. In other words:

L(theta | O) = P(O | theta)

By fixing the random variables to distinct values (representing observations), can solve for parameters (theta) through maximum likelihood estimation (MLE). MLE is analagous to the loss function used in machine learning which performs a very similar analysis. 

General MLE is an optimization problem that aims to produce solution equations that would work for any specifications of the random variables.

The same idea applies across both discrete and continuous space, in the continous space however we would use a probability density function, f(O | theta) rather than a distinct probability (due to probability laws involving sets). 

When evaluating and selecting ARIMA models by grid search, use AIC which measures number of parameters used and MLE to compute, compare many ARIMA models. 